# Preparación de los datos para análisis y Machine Learning
En este notebook crearemos un *pipeline* de preprocesamiento de texto similar al visto al principio del estudio, pero más avanzado y haciendo uso de librerías como *spaCy* y *textacy*. Una vez completado, se obtendrá un texto limpio y tokenizado listo para su análisis.

Para este caso, se va a hacer uso del dataset creado en el apartado anterior, con más de 2000 comentarios del repoositorio *zigbee2mqtt*.

Al igual que en los cuadernos anteriores, comenzaremos cargando unos ajustes predefinidos para la ejecución del entorno virtual de python.

In [11]:
import sys, os

#Carga del archivo setup.py
%run -i ../pyenv_settings/setup.py

#Imports y configuraciones de gráficas
%run "$BASE_DIR/pyenv_settings/settings.py"

#Reset del entorno virtual al iniciar la ejecución
#%reset -f

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

You are working on a local system.
Files will be searched relative to "..".


## Carga de los datos en Pandas
Cargaremos el dataset creado anteriormente con todos los comentarios de un repositorio de Github en Pandas, concretamente el archivo .csv (hay dos idénticos, uno en formato .csv y otro en .json)

In [12]:
#Ruta del archivo
file_path = "../data/output.csv"

#Carga del archivo en un DataFrame
df = pd.read_csv(file_path)

Antes de empezar a trabajar con los datos, revisaremos el nombre de las columnas y se cambiarán por otros nombres más genéricos en caso de considerarse necesario para una mejor coprensión y maniobrabilidad con el documento.

In [13]:
print(df.columns)

Index(['url', 'html_url', 'issue_url', 'id', 'node_id', 'user', 'created_at',
       'updated_at', 'author_association', 'body', 'reactions',
       'performed_via_github_app'],
      dtype='object')


Para el renombramiento de las columnas, definiremos un diccionario *column_mapping* en el que cada entrada corresponderá con el nombre de la columna original y el nuevo que se le dará. 

Si se considera que algunas columnas no son necesarias para el análisis, se pueden descartar nombrándolas como *None* o directamente sin incluirlas en el diccionario.

Viendo las columnas con las que cuenta el DataFrame se ve a simple vista que hay algunas columnas irrelevantes para el estudio, como las URLs, node_id, fechas de creación y actualización del post, asociaciones y la columna "performed_via_github_app". Estas serán descartadas a continuación sin incluirlas en el diccionario:

In [14]:
column_mapping = {
    'id' : 'id',
    'user' : 'user',
    'body' : 'text',
    'reactions' : None,
    'url' : None,
    'html_url' : None,
    'issue_url' : None,
    'node_id' : None,
    'created_at' : None,
    'update_at' : None,
    'author_association' : None,
    'performed_via_github_app' : None
}

#Se definen las columnas que se mantendrán
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]

#Seleccionar y renombrar las columnas
df = df[columns].rename(columns=column_mapping)

#Muestra de una entrada para comprobar que se ha ejecutado correctamente
df.sample(1).T

,1714
id,1138986523
user,"{'login': 'mdegat01', 'id': 2037026, 'node_id': 'MDQ6VXNlcjIwMzcwMjY=', 'avatar_url': 'https://private-avatars.githubusercontent.com/u/2037026?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJn..."
text,"@Koenkk sorry for the delay, I just tried edge and it doesn't seem to be working for me either. This PR was targetted at dev branch, was that the correct one for this? Sorry I'm not familiar with ..."
